In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time


def v_to_w(u, v, dx, dy):
    w = np.zeros_like(u)
    w[1:-1, 1:-1] = (v[1:-1, 2:] - v[1:-1, :-2]) / (2*dx) - \
        (u[2:, 1:-1] - u[:-2, 1:-1]) / (2*dy)
    return w


def w_to_phi(w, dx, dy, tol):
    psi = np.zeros_like(w)
    change = 1.0
    times = 0
    while change > tol:
        psi_new = psi.copy()
        psi_new[1:-1, 1:-1] = (w[1:-1, 1:-1] + (psi[2:, 1:-1] + psi[:-2, 1:-1]) / (dy**2)
                               + (psi[1:-1, 2:] + psi[1:-1, :-2]) / (dx**2)) / (2 / (dx**2)+2 / (dy**2))
        change = np.max(np.abs(psi_new - psi))
        psi = psi_new
        times += 1
        if times >= 10000:
            break
    return psi


def boundary_w(w, u, v, psi, dx, dy):
    w_new = w.copy()
    w_new[:, 0] = 2 / (dx**2) * (psi[:, 0] - psi[:, 1]) - 2 / dx * v[:, 0]
    w_new[:, -1] = 2 / (dx**2) * (psi[:, -1] - psi[:, -2]) + 2 / dx * v[:, -1]
    w_new[0, :] = 2 / (dy**2) * (psi[0, :] - psi[1, :]) + 2 / dy * u[0, :]
    w_new[-1, :] = 2 / (dy**2) * (psi[-1, :] - psi[-2, :]) - 2 / dy * u[-1, :]
    return w_new


def next_w(w, psi, dx, dy, dt, nu):
    w_next = w.copy()
    uw_x = -(psi[2:, 1:-1] - psi[:-2, 1:-1]) * \
        (w[1:-1, 2:] - w[1:-1, :-2]) / (4*dx*dy)
    vw_y = (psi[1:-1, 2:] - psi[1:-1, :-2]) * \
        (w[2:, 1:-1] - w[:-2, 1:-1]) / (4*dx*dy)
    ddw_y = (w[2:, 1:-1] + w[:-2, 1:-1] - 2 * w[1:-1, 1:-1]) / (dy**2)
    ddw_x = (w[1:-1, 2:] + w[1:-1, :-2] - 2 * w[1:-1, 1:-1]) / (dx**2)
    w_next[1:-1, 1:-1] = w[1:-1, 1:-1] + dt * \
        (uw_x + vw_y + nu * ddw_y + nu * ddw_x)
    return w_next


def psi_to_v(psi, u, v, dx, dy):
    u_new = u.copy()
    v_new = v.copy()
    u_new[1:-1, 1:-1] = (psi[2:, 1:-1]-psi[:-2, 1:-1]) / (2*dy)
    v_new[1:-1, 1:-1] = (psi[1:-1, 2:]-psi[1:-1, :-2]) / (2*dx)
    u_scalar = np.sqrt(u_new**2 + v_new**2)
    return u_new, v_new, u_scalar


def paint(u, x, y, name: str):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(u, cmap='viridis',
               interpolation='nearest')
    plt.xticks(np.linspace(0, u.shape[0]-1,
               num=5), np.linspace(0.0, 1.0, num=5))
    plt.yticks(np.linspace(0, u.shape[1]-1,
               num=5), np.linspace(0.0, 1.0, num=5))
    plt.colorbar()

    plt.subplot(1, 2, 2)
    plt.contour(x, y, u, cmap='viridis')
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.gca().set_aspect(1)

    plt.savefig("./HW5_fig/"+name)

    plt.show()


def paint_contour(u, x, y):
    plt.contour(x, y, u, cmap='viridis')
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
Nx = 101
Ny = 101
Lx = Ly = 1
x = np.linspace(0, Lx, Nx)
dx = x[-1] - x[-2]
y = np.linspace(0, Ly, Ny)
dy = y[-1] - y[-2]
nu = 0.0001
dt = 0.002

xx, yy = np.meshgrid(x, y)

u_b = np.zeros_like(xx)
v_b = np.zeros_like(yy)
u_b[0, :] = 16 * x ** 2 * (1.0 - x) ** 2

w_0 = v_to_w(u_b, v_b, dx, dy)
psi_0 = w_to_phi(w_0, dx, dy, 1e-4)
w_0 = boundary_w(w_0, u_b, v_b, psi_0, dx, dy)

w = w_0.copy()
psi = psi_0.copy()

for iter in range(50001):
    w = boundary_w(w, u_b, v_b, psi, dx, dy)
    w = next_w(w, psi, dx, dy, dt, nu)
    psi = w_to_phi(w, dx, dy, 1e-4)
    if iter % 1000 == 0:
        print(iter)
        velo_x, velo_y, velo = psi_to_v(psi, u_b, v_b, dx, dy)
        # paint(velo[1:-1, 1:-1])
        paint(velo[1:-1, 1:-1], xx[1:-1, 1:-1], yy[1:-1, 1:-1],f"Re_1e4_iter_{iter}")
# velo_x, velo_y, velo = psi_to_v(psi, u_b, v_b, dx, dy)
# paint(velo)